In [1]:
import sys
sys.path.append("../")


In [2]:
from sqlmodel import Session, select
from models.pps_history import PricePerShareHistory
import pandas as pd
from core.db import engine

session = Session(engine)

query = select(PricePerShareHistory)
df = pd.read_sql(query, session.bind)
df

OperationalError: (psycopg.OperationalError) connection failed: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
def calculate_roi(after: float, before: float, days: int) -> float:
    # calculate our annualized return for a vault
    pps_delta = (after - before) / (before or 1)
    annualized_roi = (1 + pps_delta) ** (365.2425 / days) - 1
    return annualized_roi

In [ ]:
from datetime import datetime, timedelta

# filter df that vault_id = 'd89eec0e-0850-4baf-ab24-53039ab47d0a'
# parse df['vault_id'] to string
df['vault_id'] = df['vault_id'].apply(str)
df1 = df[df['vault_id'] == '65f75bd7-a2d2-4764-ae31-78e4bb132c62'].sort_values(by='datetime')
df1 = df1.reset_index(drop=True)
# df1

# foreach row in df1, calculate the roi
for i, row in df1.iterrows():
    if i == 0:
        continue
    after = df1.iloc[i]['price_per_share']
    # before is price per share of 30 days ago
    before = df1.iloc[0]['price_per_share']
    days = (df1.iloc[i]['datetime'] - df1.iloc[0]['datetime']).days
    df1.at[i, 'roi'] = calculate_roi(after, before, days) * 100

df1

,datetime,price_per_share,id,vault_id,roi
0,2024-01-31,1.000000,d60c4880-865c-4f04-8ba1-4d1c86fd7c2c,d89eec0e-0850-4baf-ab24-53039ab47d0a,NaN
1,2024-02-09,1.000000,7f633834-0f5d-47c2-8296-d2d04aba6dca,d89eec0e-0850-4baf-ab24-53039ab47d0a,0.000000
2,2024-02-16,1.043481,610240a4-32fa-4da9-9ae2-2e0f07bcb43f,d89eec0e-0850-4baf-ab24-53039ab47d0a,164.215843
3,2024-02-23,1.066503,9e7fcf97-dde1-4d3c-9c1f-cac03a17675b,d89eec0e-0850-4baf-ab24-53039ab47d0a,177.997501
4,2024-03-01,1.151802,2c55bea4-4524-4934-8cf5-76273f03a925,d89eec0e-0850-4baf-ab24-53039ab47d0a,458.804269
5,2024-03-08,1.337170,a8e5f4c7-7552-4dc4-a641-ed5de8fd65d5,d89eec0e-0850-4baf-ab24-53039ab47d0a,1660.520278
6,2024-03-15,1.303140,aed6fe04-785e-4f07-8a44-45a611390a74,d89eec0e-0850-4baf-ab24-53039ab47d0a,800.610486
7,2024-03-22,1.205150,cfebf2a8-6132-4e41-8d76-5f12e428155e,d89eec0e-0850-4baf-ab24-53039ab47d0a,280.526940
8,2024-03-29,1.233630,3f3e25a1-010e-42aa-aedc-112c7892763a,d89eec0e-0850-4baf-ab24-53039ab47d0a,275.160750
